In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [7]:
dataset1=pd.read_csv("all_tickets.csv")
dataset2=pd.read_excel("MERGED_NARROW_Dedupes4.xlsx")

In [8]:
dataset1=dataset1[["body","category"]].copy()
dataset1=dataset1.reset_index()
dataset1=dataset1[dataset1.category.isin([4,5,6])].copy()#Retaining only three assignment group
dataset1.columns=["number","description","assignment_group"]

dataset1.to_csv("dataset1.csv",index=False)

In [3]:
#Treatment of Dataset2
dataset2.head()

Ticket                    Address  \
0  SR1072495                    #blank#   
1  SR1072509  6355 Richmond Avenue East   
2  SR1072566       6897 Financial Drive   
3  SR1072642        440 Glover Road – P   
4  SR1072887            246 Main Street   

                                               Title  \
0       Job IS_SECURITY_PROJ_TICKET_APP_SM59, Step 1   
1                                Computer logged out   
2                                     Password Reset   
3                Preprocess Transactions - Urgent P2   
4  Hernando Otalora - mlaf specialists manufactur...   

                                         Description Source  \
0                                            #blank#  Email   
1  One my colleagues Nolan Tutkaluk has looged hi...  Email   
2  Hello, \r\r\rMy password has expired and I nee...  Email   
3  Good day team, \r\r\rWe need the following ord...  Email   
4  please add herrando otalora to mlaf specialist...  Email   

                                            MA_Title  \
0  SEC - Job IS_SECURITY_PROJ_TICKET_APP_SM59, St...   
1                           SD - Computer logged out   
2                                SD - Password Reset   
3           SD - Preprocess Transactions - Urgent P2   
4  SD - Hernando Otalora - mlaf specialists manuf...   

                                      MA_Description MA_Support_Group  \
0                                            #blank#         Security   
1  You can reach us at 905-285-6199 or the 1-888 ...     Service Desk   
2                               password reset done.     Service Desk   
3  Good morning, \r\r\rI see that PO is dated 230...     Service Desk   
4                                            #blank#     Service Desk   

  MA_Mis_Assigned_Support_Group             Person           Team  
0                            No          SVCBATCH.            Bot  
1                            No  Gruetzner, Oliver  Manufacturing  
2                            No    Lalonde, Daniel          Sales  
3                            No   Anderson, Stacey  Manufacturing  
4                            No  Zelinsky, Sandi V  Manufacturing

In [4]:
dataset2=dataset2[dataset2.Description!="#blank#"].copy()

In [5]:
import string, re

def anonymize(s):
    s=re.sub(r"(\d{1,7}-\d{1,9}-?\d{1,9}?)","PHONEX",s)
    s=re.sub(r"([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)","EMAILIDX",s)
    s=re.sub(r'(www.(?:[-\w.]|(?:%[\da-zA-Z]{2}))+)|(https?:\/\/(?:[-\w.]|(?:%[\da-zA-Z]{2}))+)',"HTTPX",s)
    s=s.replace("mapleleaf.com","HTTPX")
    #for punctuation in string.punctuation:
    #    s=s.replace(punctuation," ")
    s=s.encode("ascii","ignore").decode("ascii")
    s=s.replace("\r"," ")
    s=s.replace("\n"," ")
    return s#.lower()

In [6]:
#Majority of tickets created from email
def getProblemStatement(s):
    
    if s.find("Subject:")>0:
        startEmail=max(s.find("From:"),0)
        if startEmail>25:
            s=s[0:startEmail]
        else:
            startEmail=s.find("Subject:")+8
            s=s[startEmail:]           
            endEmail=max(max(s.lower().find("thank"),0),max(s.lower().find("regard"),0))
            #print("1",endEmail,s.lower().find("thank"),s.lower().find("regard"),max(s.lower().find("from: "),0))
            if endEmail>max(s.lower().find("from: "),0):
                endEmail=s.lower().find("from: ")
            if endEmail==0:
                endEmail=250
            s=s[0:endEmail]
        return s
    else:
        return s#cleanText(s)#[0:startEmail])


In [7]:
dataset2["CleanDescription"]=dataset2.apply(lambda x: getProblemStatement(x.Description),axis=1)
dataset2["CleanDescription"]=dataset2.apply(lambda x: anonymize(x.CleanDescription),axis=1)
outData=dataset2[["Ticket","CleanDescription","MA_Support_Group"]].copy()
outData.columns=["number",'description','assignment_group']

In [8]:
retainedGroup=list(outData.groupby("assignment_group").description.count().reset_index().sort_values("description",ascending=False)[:10].assignment_group.values)
retainedGroup#Retaining only 10 assignment groups

['Service Desk',
 'Security',
 'Client Support',
 'IS Purchasing',
 'HCM System Support',
 'Technical Services',
 'Order Management',
 'Recipe Development',
 'MFGSCP',
 'Business Support Services']

In [9]:
outData[outData.assignment_group.isin(retainedGroup)].to_csv("dataset2.csv",index=False)